<!-- #### Example: Why Cross-Entropy is optimal -->
#### 示例：为什么交叉熵是最优的
<!-- - for bianry classification, $t_i$ is either 0 or 1.
- interpret the output $f(x_i)$ as the prob of the true value being 1 - 把模型的输出$f(x_i)$解释为真值为1的概率 -->
- 对于一个二分类问题，$t_i$只能是0或1。我们将模型的输出$f(x_i)$解释为真值为1的概率。

> $$\begin{align*}
> P(1|f(x_i)) &= f(x_i) \\
> P(0|f(x_i)) &= 1 - f(x_i) \\
> P(t_i|f(x_i)) &= f(x_i)^{t_i} (1 - f(x_i))^{(1-t_i)} \qquad \text{Since } t_i \in \{0, 1\} \\
> \log P(\{t_i\}|f) &= \sum_{i} t_i \log f(x_i) + (1-t_i) \log(1 - f(x_i)) \\
> f_{\text{ML}} &= \argmax_{f\in H} \sum_{i} t_i \log f(x_i) + (1-t_i) \log(1 - f(x_i)) \\
> \text{可以} &\text{ 被推广到多分类问题： } \\
> f_{\text{ML}} &= \argmax_{f\in H} \sum_{i} \sum_{j} t_{ij} \log f_j(x_i) \\
> \end{align*}$$

<!-- - **Key**: Minimizing Cross-Entropy is optimal for binary classification tasks. -->
- **关键点**：最小化交叉熵对于二分类任务是最优的。

<!-- - If we consider $p_i = \left<t_i, 1-t_i\right>$ and $q_i = \left<f(x_i), 1-f(x_i)\right>$, then the cross-entropy can be written as: -->
- 如果我们考虑 $p_i = \left<t_i, 1-t_i\right>$ 和 $q_i = \left<f(x_i), 1-f(x_i)\right>$，那么交叉熵可以写成：
$$\begin{align*}
\log P(\{t_i\}|f) &= \sum_{i} t_i \log f(x_i) + (1-t_i) \log(1 - f(x_i)) \\
&= \sum_{i}[(t_i(\log f(x_i) - \log(t_i)) + (1 - t_i) (\log (1-f(x_i)) - \log(1-t_i))  \\
& \qquad \qquad \qquad -(-t_i \log(t_i) - (1-t_i) \log(1-t_i) ))] \\
&= \sum_{i} D_{KL}(p_i||q_i) - H(p_i)
\end{align*}$$

<!-- - since $H(p_i)$ is fixed, minimizing cross-entropy is equivalent to minimizing $\sum_i D_{KL}(p_i||q_i)$
- 对于分类问题，使用交叉熵损失函数的一个优势是：loss is unbounded, which can help the model to learn faster. -->
- 由于 $H(p_i)$ 是固定的，最小化交叉熵等价于最小化 $\sum_i D_{KL}(p_i||q_i)$。
- 对于分类问题，使用交叉熵损失函数的一个优势是：损失函数是无界的，这可以帮助模型更快地学习。

In [ ]:
# 一个代码案例
import numpy as np
import torch
import torch.nn.functional as F

# ====================== 1. 手动实现交叉熵计算（基础版） ======================
def soft_max(logits):
    """
    手动实现Softmax函数
    :param logits: 输入数组，shape=(batch_size, num_classes)
    :return: Softmax概率分布，shape=(batch_size, num_classes)
    """
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # 减去最大值防止溢出
    probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    return probs
def cross_entropy_manual(logits, target):
    """
    手动计算交叉熵（支持批量）
    :param logits: 模型原始输出（未经过Softmax），shape=(batch_size, num_classes)
    :param target: 真实标签（独热编码），shape=(batch_size, num_classes)
    :return: 平均交叉熵损失
    """
    # Step 1: Softmax转换为概率分布（防止数值溢出，减去每行最大值）
    probs = soft_max(logits)
    # Step 2: 计算交叉熵（逐样本）
    # 加极小值防止log(0)
    cross_entropy_per_sample = -np.sum(target * np.log(probs + 1e-10), axis=1)
    # Step 3: 返回批量平均值
    return np.mean(cross_entropy_per_sample)

# ====================== 2. 固定参数设置（可自定义修改） ======================
# 模型输出（logits）：假设是2个样本，3分类任务
model_logits = np.array([
    [2.0, 1.0, 0.1],  # 样本1的模型输出
    [0.5, 2.5, 1.0]   # 样本2的模型输出
])

# 真实标签（两种形式）：
# 形式1：独热编码（手动计算用）
target_one_hot = np.array([
    [1, 0, 0],  # 样本1真实类别：第0类
    [0, 1, 0]   # 样本2真实类别：第1类
])
# 形式2：类别索引（PyTorch API用）
target_index = torch.tensor([0, 1])

# ====================== 3. 计算结果对比 ======================
# 手动实现计算
probs = soft_max(model_logits)
ce_manual = cross_entropy_manual(model_logits, target_one_hot)
print("=== 手动实现交叉熵 ===")
print(f"模型输出logits:\n{model_logits}")
print(f"真实标签（独热）:\n{target_one_hot}")
print(f"Softmax概率分布:\n{probs}")
print(f"交叉熵损失值: {ce_manual:.4f}\n")

# PyTorch API计算（推荐实际项目使用）
# 注意：F.cross_entropy 内部自动做 Softmax + 交叉熵，输入直接用logits
model_logits_torch = torch.tensor(model_logits, dtype=torch.float32)
probs_torch = F.softmax(model_logits_torch, dim=1)
ce_torch = F.cross_entropy(model_logits_torch, target_index)
print("=== PyTorch API交叉熵 ===")
print(f"模型输出logits:\n{model_logits_torch}")
print(f"真实标签（索引）: {target_index}")
print(f"Softmax概率分布:\n{probs_torch}")
print(f"交叉熵损失值: {ce_torch.item():.4f}")

=== 手动实现交叉熵 ===
模型输出logits:
[[2.  1.  0.1]
 [0.5 2.5 1. ]]
真实标签（独热）:
[[1 0 0]
 [0 1 0]]
Softmax概率分布:
[[0.65900114 0.24243297 0.09856589]
 [0.09962365 0.73612472 0.16425163]]
交叉熵损失值: 0.3617

=== PyTorch API交叉熵 ===
模型输出logits:
tensor([[2.0000, 1.0000, 0.1000],
        [0.5000, 2.5000, 1.0000]])
真实标签（索引）: tensor([0, 1])
Softmax概率分布:
tensor([[0.6590, 0.2424, 0.0986],
        [0.0996, 0.7361, 0.1643]])
交叉熵损失值: 0.3617
